In [84]:
import json
import os
import shutil

# 디렉토리 경로 설정
label_directory = './dataset/yolo_dataset/ai_hub_data/labels_json'
image_directory = './dataset/yolo_dataset/ai_hub_data/images'
modify_label_directory = './dataset/yolo_dataset/modify_data/labels/'
modify_image_directory = './dataset/yolo_dataset/modify_data/images/'

# 변경할 디렉토리가 존재하지 않으면 생성
if not os.path.exists(modify_label_directory):
    os.makedirs(modify_label_directory)
if not os.path.exists(modify_image_directory):
    os.makedirs(modify_label_directory)

# 테스트를 위한 최대 저장개수 설정
# file_count = 0
# max_files = 10

# 아우터가 포함된 이미지 번호 리스트
outer_list = []
# object 목록
class_index = {0: '하의', 1: '원피스', 2: '상의', 3: '아우터'}

# json파일의 렉트좌표를 yolo label 형식으로 변경하는 함수
def bbox_ratio(data, idx):
    img_width = data['이미지 정보']['이미지 너비']
    img_height = data['이미지 정보']['이미지 높이']
    xy = data['데이터셋 정보']['데이터셋 상세설명'].get('렉트좌표', None)
    
    x1 = round(xy[idx][0]['X좌표'] / img_width, 5)
    y1 = round(xy[idx][0]['Y좌표'] / img_height, 5)
    x2 = round(xy[idx][0]['가로'] / img_width, 5)
    y2 = round(xy[idx][0]['세로'] / img_height, 5)

    return x1, y1, x2, y2

##### 라벨링 파일 수정
for filename in os.listdir(label_directory):
    if filename.endswith('.json'):
        filepath = os.path.join(label_directory, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

            # '데이터셋 정보'가 있을때
            if '데이터셋 정보' in json_data: 
                # filename에서 '.json' 제외해주기
                file_number = filename.replace('.json', ' ')
                # 렉트좌표값 가져오기
                xy = json_data['데이터셋 정보']['데이터셋 상세설명'].get('렉트좌표', None)

                # 아우터가 포함되지 않았을 때
                if len(xy['아우터'][0]) == 0:
                    # file_count += 1
                    # yolo label 데이터를 저장할 txt파일 경로
                    modify_file_path = modify_label_directory + str(file_number) + '.txt'
                    with open(modify_file_path, 'w', encoding='utf-8') as fw:
                        # 하의, 원피스, 상의 중 어떤 객체가 있는지 확인
                        for i in range(3):
                            if len(xy[class_index[i]][0]) != 0:
                                bbox = bbox_ratio(json_data, class_index[i])
                                fw.write(str(i) + ' ')
                                for j in bbox:fw.write(str(j) + ' ')
                                fw.write("\n")
                else:
                    outer_list.append(file_number)
    
    # if file_count >= max_files:
    #     break


#### 이미지 파일 수정
for filename in os.listdir(image_directory):
    image_num = filename.replace('.jpg', ' ')
    
    # 파일 이름이 outer_list에 포함되지 않는 경우
    if image_num not in outer_list:
        # 원본 파일 경로
        source_file = os.path.join(image_directory, filename)
        # 복사할 파일 경로
        dest_file = os.path.join(modify_image_directory, filename)
        # 파일 복사
        shutil.copy(source_file, dest_file)

print("데이터셋 수정")

데이터셋 수정
